### Reference
- [Smart Geese Trained by Reinforcement Learning](https://www.kaggle.com/yuricat/smart-geese-trained-by-reinforcement-learning)
- [Alpha Zero General](https://github.com/suragnair/alpha-zero-general)

In [ ]:
%%writefile submission.py
import pickle
import bz2
import base64
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time
from copy import deepcopy
from kaggle_environments.envs.hungry_geese.hungry_geese import Action, translate
from kaggle_environments.helpers import histogram

In [ ]:
%%writefile -a submission.py
PARAM = b'XXXXXXXXXX'

In [ ]:
%%writefile -a submission.py
class MCTS():
    def __init__(self, game, nn_agent, eps=1e-8, cpuct=1.0):
        self.game = game
        self.nn_agent = nn_agent
        self.eps = eps
        self.cpuct = cpuct
        self.gamma = 0.8

        self.Qsa = {}  # stores Q values for s,a (as defined in the paper)
        self.Nsa = {}  # stores #times edge s,a was visited
        self.Ns = {}  # stores #times board s was visited
        self.Ps = {}  # stores initial policy (returned by neural net)
        self.Vs = {}  # stores value for board s (returned by neural net)
        self.Ms = {}  # stores game.getValidMoves for board s

        self.last_obs = None
        self.rewards = [0] * 4

    def getActionProb(self, obs, timelimit=1.0):
        start_time = time.time()
        while time.time() - start_time < timelimit:
            self.search(obs, self.last_obs, self.rewards[:])
            
        for i in range(4):
            if len(obs.geese[i]) > 0:
                self.rewards[i] += 100
            elif self.last_obs is not None:
                self.rewards[i] += len(self.last_obs.geese[i])

        s = self.game.stringRepresentation(obs)
        i = obs.index
        counts = [
            self.Nsa[(s, i, a)] if (s, i, a) in self.Nsa else 0
            for a in range(self.game.getActionSize())
        ]
        print(counts)

        valids = np.array(self.game.getValidMoves(obs, self.last_obs, i))
        valid_counts = counts * valids
        sum_valid_counts = np.sum(valid_counts)
        if sum_valid_counts > 0:
            prob = valid_counts / sum_valid_counts
        else:
            valids = np.array(self.game.getNonOppositeMoves(obs, self.last_obs, i))
            valid_counts = counts * valids
            sum_valid_counts = np.sum(valid_counts)
            if sum_valid_counts > 0:
                prob = valid_counts / sum_valid_counts
            else:
                prob = [1.] * valids
                prob /= np.sum(prob)

        self.last_obs = obs
        return prob
    
    def get_values(self, obs, rewards):
        '''
        rank 1:    1
        rank 1.5:  0.666
        rank 2:   -1
        rank 2.5: -1
        rank 3:   -1
        rank 3.5: -1
        rank 4:   -1
        '''
        ranks = [1] * 4
        for i in range(4):
            for j in (j for j in range(4) if j != i):
                if rewards[i] < rewards[j]:
                    ranks[i] += 1
                elif rewards[i] == rewards[j]:
                    ranks[i] += 0.5
        values = [(5 - 2 * rank) / 3 if rank < 2 else -1 for rank in ranks]
        return values

    def search(self, obs, last_obs, rewards):
        s = self.game.stringRepresentation(obs)

        for i in range(4):
            if len(obs.geese[i]) > 0:
                rewards[i] += 100
            elif last_obs is not None:
                rewards[i] += len(last_obs.geese[i])

        if s not in self.Ns:
            values = self.get_values(obs, rewards)
#             print(values)

            for i in range(4):
                if len(obs.geese[i]) == 0 or obs.step == 199:
                    continue

                # leaf node
#                 if i == obs.index:
                self.Ps[(s, i)], values[i] = self.nn_agent.predict(obs, last_obs, i)
#                     print(values[i])
#                 else:
#                     self.Ps[(s, i)] = np.array([0.25, 0.25, 0.25, 0.25])

                valids = self.game.getValidMoves(obs, last_obs, i)
                self.Ps[(s, i)] = self.Ps[(s, i)] * valids  # masking invalid moves
                sum_Ps_s = np.sum(self.Ps[(s, i)])
                if sum_Ps_s > 0:
                    self.Ps[(s, i)] /= sum_Ps_s  # renormalize

                self.Vs[(s, i)] = values[i]
                self.Ms[(s, i)] = valids
                self.Ns[s] = 0

            return values

        best_acts = [None] * 4
        for i in range(4):
            if len(obs.geese[i]) == 0:
                continue

            # pick the action with the highest upper confidence bound
            valids = self.Ms[(s, i)]
            u = np.array([-float('inf')] * 4)
            for a in range(self.game.getActionSize()):
                if valids[a]:
                    if (s, i, a) in self.Qsa:
                        u[a] = self.Qsa[(s, i, a)] + self.cpuct * self.Ps[(s, i)][a] * math.sqrt(self.Ns[s]) / (1 + self.Nsa[(s, i, a)])
                    else:
                        u[a] = self.Vs[(s, i)] + self.cpuct * self.Ps[(s, i)][a] * math.sqrt(self.Ns[s] + self.eps)

            # random choice among the best actions
            best_acts[i] = self.game.actions[np.random.choice(np.argwhere(u == u.max()).flatten())]

        next_obs = self.game.getNextState(obs, last_obs, best_acts)
        values = self.search(next_obs, obs, rewards)

        for i in range(4):
            if len(obs.geese[i]) == 0:
                continue

            a = self.game.actions.index(best_acts[i])
            v = values[i]
            if (s, i, a) in self.Qsa:
                self.Qsa[(s, i, a)] = (self.Nsa[(s, i, a)] * self.Qsa[(s, i, a)] + v) / (self.Nsa[(s, i, a)] + 1)
                self.Nsa[(s, i, a)] += 1
            else:
                self.Qsa[(s, i, a)] = v
                self.Nsa[(s, i, a)] = 1

        self.Ns[s] += 1
        
        values = [value * self.gamma for value in values]
        return values

In [ ]:
%%writefile -a submission.py
class HungryGeese(object):
    def __init__(self,
                 rows=7,
                 columns=11,
                 actions=[Action.NORTH, Action.SOUTH, Action.WEST, Action.EAST],
                 hunger_rate=40):
        self.rows = rows
        self.columns = columns
        self.actions = actions
        self.hunger_rate = hunger_rate

    def getActionSize(self):
        return len(self.actions)

    def getNextState(self, obs, last_obs, directions):
        next_obs = deepcopy(obs)
        next_obs.step += 1
        geese = next_obs.geese
        food = next_obs.food
        
        for i in range(4):
            goose = geese[i]
            
            if len(goose) == 0: 
                continue
            
            head = translate(goose[0], directions[i], self.columns, self.rows)

            # Check action direction
            if last_obs is not None and head == last_obs.geese[i][0]:
                geese[i] = []
                continue

            # Consume food or drop a tail piece.
            if head in food:
                food.remove(head)
            else:
                goose.pop()
            
            # Add New Head to the Goose.
            goose.insert(0, head)

            # If hunger strikes remove from the tail.
            if next_obs.step % self.hunger_rate == 0:
                if len(goose) > 0:
                    goose.pop()

        goose_positions = histogram(
            position
            for goose in geese
            for position in goose
        )

        # Check for collisions.
        for i in range(4):
            if len(geese[i]) > 0:
                head = geese[i][0]
                if goose_positions[head] > 1:
                    geese[i] = []
        
        return next_obs

    def getValidMoves(self, obs, last_obs, index):   
        geese = obs.geese
        pos = geese[index][0]
        obstacles = {position for goose in geese for position in goose[:-1]}
        if last_obs is not None: obstacles.add(last_obs.geese[index][0])
        
        valid_moves = [
            translate(pos, action, self.columns, self.rows) not in obstacles
            for action in self.actions
        ]

        return valid_moves
    
    def getNonOppositeMoves(self, obs, last_obs, index):   
        geese = obs.geese
        pos = geese[index][0]
        opposite = None
        if last_obs is not None: opposite = last_obs.geese[index][0]

        valid_moves = [
            translate(pos, action, self.columns, self.rows) != opposite
            for action in self.actions
        ]

        return valid_moves

    def stringRepresentation(self, obs):      
        return str(obs.geese + obs.food)

In [ ]:
%%writefile -a submission.py
# Neural Network for Hungry Geese
class TorusConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.edge_size = (kernel_size[0] // 2, kernel_size[1] // 2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size, padding=self.edge_size, padding_mode='circular')
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = self.conv(x)
        h = self.bn(h) if self.bn is not None else h
        return h


class GeeseNet(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 12, 32
        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])
        self.head_p = nn.Linear(filters, 4, bias=False)
        self.head_v = nn.Linear(filters * 2, 1, bias=False)

    def forward(self, x):
        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))
        h_head = (h * x[:,:1]).view(h.size(0), h.size(1), -1).sum(-1)
        h_avg = h.view(h.size(0), h.size(1), -1).mean(-1)
        p = torch.softmax(self.head_p(h_head), 1)
        v = torch.tanh(self.head_v(torch.cat([h_head, h_avg], 1)))

        return p, v


class NNAgent():
    def __init__(self, state_dict):
        self.model = GeeseNet()
        self.model.load_state_dict(state_dict)
        self.model.eval()
        
    def predict(self, obs, last_obs, index):
        x = self._make_input(obs, last_obs, index)
        with torch.no_grad():
            xt = torch.from_numpy(x).unsqueeze(0)
            p, v = self.model(xt)
            
        return p.squeeze(0).detach().numpy(), v.item()
        
    # Input for Neural Network
    def _make_input(self, obs, last_obs, index):
        b = np.zeros((17, 7 * 11), dtype=np.float32)
        
        for p, pos_list in enumerate(obs.geese):
            # head position
            for pos in pos_list[:1]:
                b[0 + (p - index) % 4, pos] = 1
            # tip position
            for pos in pos_list[-1:]:
                b[4 + (p - index) % 4, pos] = 1
            # whole position
            for pos in pos_list:
                b[8 + (p - index) % 4, pos] = 1

        # previous head position
        if last_obs is not None:
            for p, pos_list in enumerate(last_obs.geese):
                for pos in pos_list[:1]:
                    b[12 + (p - index) % 4, pos] = 1

        # food
        for pos in obs.food:
            b[16, pos] = 1

        return b.reshape(-1, 7, 11)

In [ ]:
%%writefile -a submission.py
game = HungryGeese()
state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))
agent = NNAgent(state_dict)
mcts = MCTS(game, agent)

def alphageese_agent(obs, config):
#     print(obs)
    probs = mcts.getActionProb(obs, timelimit=config.actTimeout*1.)    
#     probs = mcts.getActionProb(obs, timelimit=0.05)

    action = game.actions[np.random.choice(np.argwhere(probs == probs.max()).flatten())]
    return action.name

In [ ]:
import torch
import base64
import bz2
import pickle

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights = torch.load('../input/hungry-geese-models/latest.pth', map_location=device)
PARAM = base64.b64encode(bz2.compress(pickle.dumps(weights)))

In [ ]:
# Read in the submission file
with open('submission.py',) as file:
    filedata = file.read()

# Replace the target string
filedata = filedata.replace('XXXXXXXXXX', PARAM.decode("utf-8") )

# Write the file out again
with open('submission.py','w') as file:
    file.write(filedata)

In [ ]:
# %%writefile crazy-goose.py
# # from https://www.kaggle.com/gabrielmilan/crazy-goose
# # Base code for this from
# # https://www.kaggle.com/ilialar/risk-averse-greedy-goose

# import numpy as np
# from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col

# # Moves constants
# SOUTH = 1
# NORTH = 2
# EAST  = 3
# WEST  = 4
# REVERSE_MOVE = {
#     None : None,
#     SOUTH: NORTH,
#     NORTH: SOUTH,
#     EAST : WEST,
#     WEST : EAST,
# }
# CIRCLE_MOVE = {
#     None : None,
#     SOUTH: WEST,
#     NORTH: EAST,
#     EAST : SOUTH,
#     WEST : NORTH
# }

# # Board constants
# MY_HEAD             =  2
# FOOD_CELL           =  1
# EMPTY               =  0
# HEAD_POSSIBLE_CELL  = -1
# BODY_CELL           = -2

# # Store last move
# last_move = None
# last_eaten = 0
# last_size = 1
# step = 0

# # Returns a list of possible destinations in order to reach `dest_cell`
# def move_towards (head_cell, neck_cell, dest_cell, configuration):
#     print ("--- Computing food movements...")
#     destinations = []
#     x_head, y_head = row_col(head_cell, configuration.columns)
#     x_neck, y_neck = row_col(neck_cell, configuration.columns)
#     x_dest, y_dest = row_col(dest_cell, configuration.columns)
#     print ("-> Head at ({}, {})".format(x_head, y_head))
#     print ("-> Neck at ({}, {})".format(x_neck, y_neck))
#     print ("-> Dest at ({}, {})".format(x_dest, y_dest))
#     dx = x_head - x_dest
#     dy = y_head - y_dest
#     if (dx >= 4):
#         dx = 7 - dx
#     elif (dx <= -4):
#         dx += 7
#     if (dy >= 6):
#         dy = 11 - dy
#     elif (dy <= -6):
#         dy += 11
#     print ("dx={}, dy={}".format(dx, dy))
#     if (dx > 0):
#         x_move = (x_head - 1 + 7) % 7
#         y_move = y_head
#         print ("Move ({}, {}), Neck ({}, {})".format(x_move, y_move, x_neck, y_neck))
#         if not ((x_move == x_neck) and (y_move == y_neck)):
#             destinations.append((x_move, y_move, NORTH))
#     elif (dx < 0):
#         x_move = (x_head + 1 + 7) % 7
#         y_move = y_head
#         print ("Move ({}, {}), Neck ({}, {})".format(x_move, y_move, x_neck, y_neck))
#         if not ((x_move == x_neck) and (y_move == y_neck)):
#             destinations.append((x_move, y_move, SOUTH))
#     if (dy > 0):
#         x_move = x_head
#         y_move = (y_head - 1 + 11) % 11
#         print ("Move ({}, {}), Neck ({}, {})".format(x_move, y_move, x_neck, y_neck))
#         if not ((x_move == x_neck) and (y_move == y_neck)):
#             destinations.append((x_move, y_move, WEST))
#     elif (dy < 0):
#         x_move = x_head
#         y_move = (y_head + 1 + 11) % 11
#         print ("Move ({}, {}), Neck ({}, {})".format(x_move, y_move, x_neck, y_neck))
#         if not ((x_move == x_neck) and (y_move == y_neck)):
#             destinations.append((x_move, y_move, EAST))
#     return destinations

# def get_all_movements(goose_head, configuration):
#     x_head, y_head = row_col(goose_head, configuration.columns)
#     movements = []
#     movements.append(((x_head - 1 + 7) % 7, y_head, NORTH))
#     movements.append(((x_head + 1 + 7) % 7, y_head, SOUTH))
#     movements.append((x_head, (y_head - 1 + 11) % 11, WEST))
#     movements.append((x_head, (y_head + 1 + 11) % 11, EAST))
#     return movements
    
# def get_nearest_cells(x, y):
#     # Returns adjacent cells from the current one
#     result = []
#     for i in (-1,+1):
#         result.append(((x+i+7)%7, y))
#         result.append((x, (y+i+11)%11))
#     return result

# # Compute L1 distance between cells
# def cell_distance (a, b, configuration):
#     xa, ya = row_col(a, configuration.columns)
#     xb, yb = row_col(b, configuration.columns)
#     dx = abs(xa - xb)
#     dy = abs(ya - yb)
#     if (dx >= 4):
#         dx = 7 - dx
#     if (dy >= 6):
#         dy = 11 - dy
#     return dx + dy

# # Tells if that particular cell forbids movement on the next step
# def is_closed (movement, board):
#     return all([board[x_adj, y_adj] for (x_adj, y_adj) in get_nearest_cells(movement[0], movement[1])])

# def is_safe (movement, board):
#     return board[movement[0], movement[1]] >= 0

# def is_half_safe (movement, board):
#     return board[movement[0], movement[1]] >= -1

# def agent (obs_dict, config_dict):
#     global last_move
#     global last_eaten
#     global last_size
#     global step
#     print ("==============================================")
#     observation = Observation(obs_dict)
#     configuration = Configuration(config_dict)
#     player_index = observation.index
#     player_goose = observation.geese[player_index]
#     player_head = player_goose[0]
#     player_row, player_column = row_col(player_head, configuration.columns)

#     if (len(player_goose) > last_size):
#         last_size = len(player_goose)
#         last_eaten = step
#     step += 1
    
#     moves = {
#         1: 'SOUTH',
#         2: 'NORTH',
#         3: 'EAST',
#         4: 'WEST'
#     }

#     board = np.zeros((7, 11))
    
#     # Adding food to board
#     for food in observation.food:
#         x, y = row_col(food, configuration.columns)
#         print ("Food cell on ({}, {})".format(x, y))
#         board[x, y] = FOOD_CELL
        
#     # Adding geese to the board
#     for i in range(4):
#         goose = observation.geese[i]
#         # Skip if goose is dead
#         if len(goose) == 0:
#             continue
#         # If it's an opponent
#         if i != player_index:
#             x, y = row_col(goose[0], configuration.columns)
#             # Add possible head movements for it
#             for px, py in get_nearest_cells(x, y):
#                 print ("Head possible cell on ({}, {})".format(px, py))
#                 # If one of these head movements may lead the goose
#                 # to eat, add tail as BODY_CELL, because it won't move.
#                 if board[px, py] == FOOD_CELL:
#                     x_tail, y_tail = row_col(goose[-1], configuration.columns)
#                     print ("Adding tail on ({}, {}) as the goose may eat".format(x_tail, y_tail))
#                     board[x_tail, y_tail] = BODY_CELL
#                 board[px, py] = HEAD_POSSIBLE_CELL
#         # Adds goose body without tail (tail is previously added only if goose may eat)
#         for n in goose[:-1]:
#             x, y = row_col(n, configuration.columns)
#             print ("Body cell on ({}, {})".format(x, y))
#             board[x, y] = BODY_CELL
    
#     # Adding my head to the board
#     x, y = row_col(player_head, configuration.columns)
#     print ("My head is at ({}, {})".format(x, y))
#     board[x, y] = MY_HEAD
    
#     # Debug board
#     print (board)
    
#     # Iterate over food and geese in order to compute distances for each one
#     food_race = {}
#     for food in observation.food:
#         food_race[food] = {}
#         for i in range(4):
#             goose = observation.geese[i]
#             if len(goose) == 0:
#                 continue
#             food_race[food][i] = cell_distance(goose[0], food, configuration)
    
#     # The best food is the least coveted
#     best_food = None
#     best_distance = float('inf')
#     best_closest_geese = float('inf')
#     for food in food_race:
#         print ("-> Food on {}".format(row_col(food, configuration.columns)))
#         my_distance = food_race[food][player_index]
#         print (" - My distance is {}".format(my_distance))
#         closest_geese = 0
#         for goose_id in food_race[food]:
#             if goose_id == player_index:
#                 continue
#             if food_race[food][goose_id] <= my_distance:
#                 closest_geese += 1
#         print (" - There are {} closest geese".format(closest_geese))
#         if (closest_geese < best_closest_geese):
#             best_food = food
#             best_distance = my_distance
#             best_closest_geese = closest_geese
#             print ("  * This food is better")
#         elif (closest_geese == best_closest_geese) and (my_distance <= best_distance):
#             best_food = food
#             best_distance = my_distance
#             best_closest_geese = closest_geese
#             print ("  * This food is better")
            
#     # Now that the best food has been found, check if the movement towards it is safe.
#     # Computes every available move and then check for move priorities.
#     if len(player_goose) > 1:
#         food_movements = move_towards(player_head, player_goose[1], best_food, configuration)
#     else:
#         food_movements = move_towards(player_head, player_head, best_food, configuration)
#     all_movements = get_all_movements(player_head, configuration)
#     # Excluding last movement reverse
#     food_movements = [move for move in food_movements if move[2] != REVERSE_MOVE[last_move]]
#     all_movements  = [move for move in all_movements if move[2] != REVERSE_MOVE[last_move]]
#     print ("-> Available food moves: {}".format(food_movements))
#     print ("-> All moves: {}".format(all_movements))
    
#     # Trying to reach goal size of 4
#     if (len(player_goose) < 4):
        
#         # 1. Food movements that are safe and not closed
#         for food_movement in food_movements:
#             print ("Food movement {}".format(food_movement))
#             if is_safe (food_movement, board) and not is_closed(food_movement, board):
#                 print ("It's safe! Let's move {}!".format(moves[food_movement[2]]))
#                 last_move = food_movement[2]
#                 return moves[food_movement[2]] # Move here

#         # 2. Any movement safe and not closed
#         for movement in all_movements:
#             print ("Movement {}".format(movement))
#             if is_safe (movement, board) and not is_closed(movement, board):
#                 print ("It's safe! Let's move {}!".format(moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here

#         # 3. Food movements half safe and not closed
#         for food_movement in food_movements:
#             if is_half_safe (food_movement, board) and not is_closed(food_movement, board):
#                 print ("Food movement {} is half safe, I'm going {}!".format(food_movement, moves[food_movement[2]]))
#                 last_move = food_movement[2]
#                 return moves[food_movement[2]] # Move here

#         # 4. Any movement half safe and not closed
#         for movement in all_movements:
#             if is_half_safe (movement, board) and not is_closed(movement, board):
#                 print ("Movement {} is half safe, I'm going {}!".format(movement, moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here

#         # 5. Food movements that are safe
#         for food_movement in food_movements:
#             print ("Food movement {}".format(food_movement))
#             if is_safe (food_movement, board):
#                 print ("It's safe! Let's move {}!".format(moves[food_movement[2]]))
#                 last_move = food_movement[2]
#                 return moves[food_movement[2]] # Move here

#         # 6. Any movement safe
#         for movement in all_movements:
#             print ("Movement {}".format(movement))
#             if is_safe (movement, board):
#                 print ("It's safe! Let's move {}!".format(moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here

#         # 7. Food movements half safe
#         for food_movement in food_movements:
#             if is_half_safe (food_movement, board):
#                 print ("Food movement {} is half safe, I'm going {}!".format(food_movement, moves[food_movement[2]]))
#                 last_move = food_movement[2]
#                 return moves[food_movement[2]] # Move here

#         # 8. Any movement half safe
#         for movement in all_movements:
#             if is_half_safe (movement, board):
#                 print ("Movement {} is half safe, I'm going {}!".format(movement, moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here
    
#     # Just trying to walk in circles
#     else:
        
#         # Delete food moves
#         food_coordinates = []
#         for food in food_race:
#             x_food, y_food = row_col(food, configuration.columns)
#             food_coordinates.append((x_food, y_food))
#         available_moves = []
#         for move in all_movements:
#             for (x_food, y_food) in food_coordinates:
#                 if (move[0] != x_food) or (move[1] != y_food):
#                     available_moves.append(move)
        
#         # 1. Run in circles if you can
#         circle_move = CIRCLE_MOVE[last_move]
#         for move in available_moves:
#             if (move[2] == circle_move) and (is_safe(move, board)) and not (is_closed(move, board)):
#                 last_move = move[2]
#                 return moves[move[2]]
        
#         # 2. Any movement safe and not closed
#         for movement in all_movements:
#             print ("Movement {}".format(movement))
#             if is_safe (movement, board) and not is_closed(movement, board):
#                 print ("It's safe! Let's move {}!".format(moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here

#         # 3. Any movement half safe and not closed
#         for movement in all_movements:
#             if is_half_safe (movement, board) and not is_closed(movement, board):
#                 print ("Movement {} is half safe, I'm going {}!".format(movement, moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here

#         # 4. Any movement safe
#         for movement in all_movements:
#             print ("Movement {}".format(movement))
#             if is_safe (movement, board):
#                 print ("It's safe! Let's move {}!".format(moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here

#         # 5. Any movement half safe
#         for movement in all_movements:
#             if is_half_safe (movement, board):
#                 print ("Movement {} is half safe, I'm going {}!".format(movement, moves[movement[2]]))
#                 last_move = movement[2]
#                 return moves[movement[2]] # Move here

#     # Finally, if all moves are unsafe, randomly pick one
#     rand_pick = np.random.randint(4) + 1
#     last_move = rand_pick
#     print ("Yeah whatever, I'm going {}".format(moves[rand_pick]))
#     return moves[rand_pick]

In [ ]:
# from kaggle_environments import make
# env = make("hungry_geese", debug=True)
# env.reset()
# # env.run(['submission.py', 'crazy-goose.py', 'crazy-goose.py', 'crazy-goose.py'])
# env.run(['submission.py'] * 4)
# env.render(mode="ipython", width=700, height=600)